In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
#from torch.utils.data import Subset
from torch.utils.data import DataLoader, random_split



In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #self.bn1 = nn.BatchNorm2d(32)
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.bn2 = nn.BatchNorm2d(64)
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.dropout1 = nn.Dropout2d(0.25)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        #self.bn3 = nn.BatchNorm2d(128)
        #self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        #self.bn4 = nn.BatchNorm2d(256)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.dropout2= nn.Dropout2d(0.25)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.bn5 = nn.BatchNorm2d(512)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.bn6 = nn.BatchNorm2d(1024)
        #self.conv7 = nn.Conv2d(1024, 10, 3)

        #self.conv1 = nn.Conv2d(1, 10, 3) #input -? OUtput? RF
        #self.bn1 = nn.BatchNorm2d(10)
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.dropout1 = nn.Dropout2d(0.25)
        #self.conv3 = nn.Conv2d(10, 20, 3)
        #self.bn3 = nn.BatchNorm2d(20)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.dropout2= nn.Dropout2d(0.25)
        #self.fc1 = nn.Linear(20*5*5,32)
        #self.fc2 = nn.Linear(32, 10)

        self.conv1 = nn.Conv2d(1, 10, 3, padding=1)     # ~80 params
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, 3, padding=1)    # ~1,168 params
        self.bn2 = nn.BatchNorm2d(20)
        self.conv3 = nn.Conv2d(20, 20, 3, stride=2)    # ~2,320 params
        #self.conv3 = nn.Conv2d(20, 20, 3)    # ~2,320 params
        self.bn3 = nn.BatchNorm2d(20)

        # After 2 poolings (28→7) + stride 2 conv (7→3) → feature map = 16×3×3 = 144
        self.fc1 = nn.Linear(180, 64)                  # (144*64 + 64) = 9,280
        self.bn4 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 10)                   # 650 params


    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)  # 28→14
        #x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)  # 14→7
        #x = self.dropout(x)
        x = F.relu(self.bn3(self.conv3(x)))  # 7→3
        x = torch.flatten(x, 1)
        x = F.relu(self.bn4(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x)
        #return x

In [21]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
           Dropout-3           [-1, 10, 14, 14]               0
            Conv2d-4           [-1, 20, 14, 14]           1,820
       BatchNorm2d-5           [-1, 20, 14, 14]              40
           Dropout-6             [-1, 20, 7, 7]               0
            Conv2d-7             [-1, 20, 3, 3]           3,620
       BatchNorm2d-8             [-1, 20, 3, 3]              40
            Linear-9                   [-1, 64]          11,584
      BatchNorm1d-10                   [-1, 64]             128
          Dropout-11                   [-1, 64]               0
           Linear-12                   [-1, 10]             650
Total params: 18,002
Trainable params: 18,002
Non-trainable params: 0
---------------------------------

/tmp/ipython-input-388598364.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [22]:
# Train data transformations
train_transforms = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation((-15., 15.), fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

# Test data transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

In [ ]:
# Create a subset of the dataset with only 50K/10K
#subset_indices = list(range(50000))  # Select the 50,000 indices for training
#subset_indices_test = list(range(10000)) # Select the 10,000 indices for validation


In [23]:
torch.manual_seed(1)
#batch_size = 128
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
#train_loader = torch.utils.data.DataLoader(
   # Subset(datasets.MNIST('../data', train=True, download=True,
   #                 transform=train_transforms),subset_indices) ,
    #batch_size=batch_size, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
    #Subset(datasets.MNIST('../data', train=False, transform=test_transforms),subset_indices_test),
    #batch_size=batch_size, shuffle=True, **kwargs)
# Download MNIST
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=train_transforms)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=test_transforms)

# Split test dataset into validation (50 samples) and test (9950 samples)
val_size = 50
test_size = len(test_dataset) - val_size
val_dataset, real_test_dataset = random_split(test_dataset, [val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,**kwargs)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,**kwargs)
test_loader  = DataLoader(real_test_dataset, batch_size=batch_size, shuffle=False,**kwargs)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.05MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.08MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.44MB/s]


In [24]:
from tqdm import tqdm

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

def train(model, device, train_loader,val_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        correct += GetCorrectPredCount(output, target)
        processed += len(data)

        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    pbar.set_description(desc= f'\nTrain: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

    # Validation accuracy
    model.eval()
    correct = 0
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    #print(f"Epoch {epoch}, Validation Accuracy: {100 * correct / len(val_dataset):0.2f}%")
    val_loss /= len(val_loader.dataset)
    print(f"\n Epoch {epoch},Average loss: {val_loss:.4f},Validation Accuracy: {correct}/{len(val_loader.dataset)} ({100. * correct / len(val_loader.dataset):0.2f}%)")

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [31]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

#optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(1, 20):
    train(model, device, train_loader,val_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]/tmp/ipython-input-1638357843.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
100%|██████████| 938/938 [00:24<00:00, 38.11it/s]



 Epoch 1,Average loss: 0.0860,Validation Accuracy: 47/50 (94.00%)

Test set: Average loss: 0.0525, Accuracy: 9784/9950 (98.33%)


100%|██████████| 938/938 [00:25<00:00, 37.45it/s]



 Epoch 2,Average loss: 0.0258,Validation Accuracy: 50/50 (100.00%)

Test set: Average loss: 0.0323, Accuracy: 9849/9950 (98.98%)


100%|██████████| 938/938 [00:24<00:00, 38.89it/s]


...


 Epoch 17,Average loss: 0.0031,Validation Accuracy: 50/50 (100.00%)

Test set: Average loss: 0.0224, Accuracy: 9877/9950 (99.27%)


100%|██████████| 938/938 [00:24<00:00, 39.05it/s]



 Epoch 18,Average loss: 0.0172,Validation Accuracy: 49/50 (98.00%)

Test set: Average loss: 0.0226, Accuracy: 9876/9950 (99.26%)


100%|██████████| 938/938 [00:24<00:00, 38.96it/s]



 Epoch 19,Average loss: 0.0070,Validation Accuracy: 50/50 (100.00%)

Test set: Average loss: 0.0179, Accuracy: 9893/9950 (99.43%)
